In [1]:
import pandas as pd
import numpy as np
import os


fake_data = pd.read_csv('Panorama/metatable.csv', sep='\t')

data = []
for i in range(fake_data.shape[0]):
    file = open('Panorama/texts/'+str(i)+'.txt')
    data.append(' '.join(file.readlines()))
fake_data = pd.concat([fake_data, pd.DataFrame(data, columns=['text'])], axis=1)
fake_data['text'] = fake_data['text'].str.lower().str.replace('\s', ' ').\
                                      str.replace(r'[^a-zа-я0-9 ]', '')
fake_data['title'] = fake_data['title'].str.lower().str.replace('\s', ' ').\
                                      str.replace(r'[^a-zа-я0-9 ]', '')
fake_data = fake_data[['text', 'title', 'date', 'link']]

def get_data(news):
    data = []
    meta_data = pd.read_csv(news + '/newmetadata.csv', sep='\t').drop_duplicates()
    for name in os.listdir(news + '/texts'):
        file = open(news + '/texts/'+name)
        data.append(' '.join(file.readlines()))
    
    data = pd.DataFrame({'text': data, 'textid': os.listdir(news + '/texts')})
    meta_data = pd.concat([meta_data.sort_values(['textid']).reset_index(drop=True),
                               data.sort_values(['textid']).reset_index(drop=True)['text']], axis=1)
    meta_data['text'] = meta_data['text'].str.lower().str.replace('\s', ' ').str.replace(r'[^a-zа-я0-9 ]', '')
    meta_data['textname'] = meta_data['textname'].str.lower().str.replace('\s', ' ').str.replace(r'[^a-zа-я0-9 ]', '')
    return meta_data[['text', 'textname', 'date', 'source']].sort_values(['date']).reset_index(drop=True).\
                                                             rename({'textname':'title'}, axis=1)

interfax_data = get_data('interfax')
print(interfax_data.shape[0])

kp_data = get_data('KP')
print(kp_data.shape[0])

lenta_data = get_data('Lenta')
print(lenta_data.shape[0])

46430
45504
36446


In [2]:
eng_data = pd.read_csv('fake-news/train.csv').drop(columns=['id'])
eng_data['text'] = eng_data['text'].str.lower().str.replace('\s', ' ').str.replace(r'[^a-zа-я0-9 ]', '')
eng_data['title'] = eng_data['title'].str.lower().str.replace('\s', ' ').str.replace(r'[^a-zа-я0-9 ]', '')
eng_data.drop(index=eng_data.index[eng_data.text.isnull()], inplace=True)
print(eng_data.shape[0])

emd1_texts = pd.read_csv('all-the-news/articles1.csv')['content'].str.lower().str.replace('\s', ' ').str.replace(r'[^a-zа-я0-9 ]', '')
emd2_texts = pd.read_csv('all-the-news/articles2.csv')['content'].str.lower().str.replace('\s', ' ').str.replace(r'[^a-zа-я0-9 ]', '')
emd3_texts = pd.read_csv('all-the-news/articles3.csv')['content'].str.lower().str.replace('\s', ' ').str.replace(r'[^a-zа-я0-9 ]', '')

20761


In [3]:
dict_emb = 'eng_rus'
#dict_emb = 'rus'


texts = [lenta_data.iloc[i].text.split() for i in range(len(lenta_data))]
texts += [kp_data.iloc[i].text.split() for i in range(len(kp_data))]
texts += [interfax_data.iloc[i].text.split() for i in range(len(interfax_data))]

#texts += [eng_data.iloc[i].text.split() for i in range(len(eng_data))]

if dict_emb == 'eng_rus':
    texts += [emd1_texts.iloc[i].split() for i in range(len(emd1_texts))]
    texts += [emd2_texts.iloc[i].split() for i in range(len(emd2_texts))]
    texts += [emd3_texts.iloc[i].split() for i in range(len(emd3_texts))]

In [4]:
%%time
from gensim.models import Word2Vec



n = 64
model = Word2Vec(texts, size=n, window=10, min_count=5, workers=12)
model.save("model/word2vec_" + dict_emb + ".bin")
model = Word2Vec.load("model/word2vec_" + dict_emb + ".bin")

CPU times: user 21min 55s, sys: 8.76 s, total: 22min 4s
Wall time: 6min 28s


In [5]:
from gensim.models import LdaModel
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary


dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

n_tm = 64
model = LdaModel(corpus, num_topics = n_tm)
model.save("model/lda_" + dict_emb + ".bin")
model = LdaModel.load("model/lda_" + dict_emb + ".bin")

In [6]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


docs = []
for i in range(len(texts)):
    docs.append(TaggedDocument(texts[i], [str(i)]))
    
n_d2v = 64
model = Doc2Vec(vector_size=n_d2v, window=8, min_count=5, workers=12, alpha=0.025, min_alpha=0.01, dm=0, epochs=10)
model.build_vocab(docs)
model.train(docs, total_examples=len(docs), epochs=20)
model.save("model/doc2vec_" + dict_emb + ".bin")
model = Doc2Vec.load("model/doc2vec_" + dict_emb + ".bin")